In [1]:
!pip install ucimlrepo

In [2]:
import pandas as pd
import numpy as np
from ucimlrepo import fetch_ucirepo

fetch, clean up and separate datasets into training, validation, and testing sets at a ratio of 1:1:1

In [3]:
dataset1 = fetch_ucirepo(id=887)
X = dataset1.data.features 
y = dataset1.data.targets
df1 = pd.concat([X,y], axis = 1)
df1 = df1.dropna()
df1 = df1.drop_duplicates()
temp = np.array_split(df1, 3)
df1_train = temp[2]
df1_validate = temp[1]
df1_test = temp[0]

In [4]:
dataset2 = fetch_ucirepo(id=15)
X2 = dataset2.data.features
y2 = dataset2.data.targets
df2 = pd.concat([X2,y2], axis = 1)
df2 = df2.drop_duplicates()
df2 = df2.dropna()
temp2 = np.array_split(df2, 3)
df2_train = temp2[0]
df2_validate = temp2[1]
df2_test = temp2[2]

get a list of feature names for each dataset

In [5]:
feature_list = df1.columns.tolist()
column_list = feature_list[:]
feature_list.pop()

feature_list_2 = df2.columns.tolist()
column_list_2 = feature_list_2[:]
feature_list_2.pop()

'Class'

In [6]:
print(feature_list, feature_list_2)

['RIAGENDR', 'PAQ605', 'BMXBMI', 'LBXGLU', 'DIQ010', 'LBXGLT', 'LBXIN'] ['Clump_thickness', 'Uniformity_of_cell_size', 'Uniformity_of_cell_shape', 'Marginal_adhesion', 'Single_epithelial_cell_size', 'Bare_nuclei', 'Bland_chromatin', 'Normal_nucleoli', 'Mitoses']


define DTNode class

In [7]:
class DTNode:
    def __init__(self, data, p = None, l = None, r = None, feat = None, threshold = None):
        self.data = data
        self.l = l
        self.r = r
        self.p = p
        self.feat = feat
        self.threshold = threshold
        self.predicted_class = None
        self.confidence = None
    def get_split_data(self):
        # splits data according to the specified feature and threshold
        data_l = self.data[self.data[self.feat] < self.threshold]
        data_r = self.data[self.data[self.feat] >= self.threshold]
        return data_l, data_r
    def get_gini(self): # gini impurity of the dataset contained in the node
        y = self.data.iloc[:,-1]
        # print(y)
        class_counts = y.value_counts()
        probs = class_counts / len(y)
        gini = 1- sum(probs*probs)
        return gini
    def get_weighted_gini(self): #gini weighted on sample size, useful for calculation
        y = self.data.iloc[:,-1]
        class_counts = y.value_counts()
        probs = class_counts / len(y)
        gini = 1- sum(probs**2)
        return gini * self.get_data_size()
    def get_data_size(self):
        return self.data.shape[0]


define DT class


In [8]:
class DT:
    def __init__(self, feature_list, max_depth = 64, min_split = 3, min_gini = 0.02):
        #self.find_best_split(self.root)
        self.feature_list = feature_list
        self.max_depth= max_depth
        self.min_split = min_split
        self.min_gini = min_gini
    def fit(self, data):
        self.root = DTNode(data)
        self.build_tree(self.root,1)
    def predict(self, data):
        # returns a list of expected target values for the input dataset
        prediction_list = []
        for row_pos in range(data.shape[0]):
            prediction_list.append(self.predict_row(data.iloc[row_pos], self.root))
        return prediction_list
    def predict_row(self, row, node):
        if node.l is None and node.r is None:
            return node.predicted_class # if node is leaf node, return prediction based on most common target value
        else:
            if row[node.feat] < node.threshold: # recursively traverse the tree until a leaf node
                return self.predict_row(row, node.l) if node.l is not None else self.predict_row(row, node.r)
            else:
                return self.predict_row(row, node.r) if node.r is not None else self.predict_row(row, node.l)
    def build_tree(self, node, depth):
        if node.get_data_size() <= self.min_split or depth > self.max_depth or node.get_gini() <= self.min_gini:
            node.l = None
            node.r = None
            y = node.data.iloc[:,-1]
            node.predicted_class = y.mode()[0]
            return
        else:
            best_feature, best_threshold = self.find_best_split(node, self.feature_list)
            l,r = self.split_node(node,best_feature,best_threshold)
            self.build_tree(l, depth + 1)
            self.build_tree(r, depth + 1)
            node.l = l
            node.r = r
            return 
    def find_best_split(self, node, feature_list):
        best_cost = 99999999
        for feature in feature_list:
            for threshold in np.arange(node.data[feature].min(), node.data[feature].max(), 0.5):
                temp_node = DTNode(node.data,feat = feature,threshold = threshold)
                temp_node.l = DTNode(temp_node.get_split_data()[0])
                temp_node.r = DTNode(temp_node.get_split_data()[1])
                # calculate weighted average of gini impurities of both children as the cost
                cost = (temp_node.l.get_weighted_gini() + temp_node.r.get_weighted_gini()) / (temp_node.get_data_size())
                if cost < best_cost:
                    best_cost = cost
                    best_feature = feature
                    best_threshold = threshold
        return best_feature,best_threshold
    def split_node(self,node,feat,threshold):
        node.feat = feat
        node.threshold = threshold
        split_data_l, split_data_r = node.get_split_data()
        l = DTNode(split_data_l) if split_data_l.shape[0] > 0 else None
        r = DTNode(split_data_r) if split_data_r.shape[0] > 0 else None
        return l, r

define accuracy evaluation and validation functions


In [12]:
def eval_acc(training_data,test_data,feature_list, max_depth = 64, min_split = 1, min_gini = 0.05):
    correct_predictions = 0
    TestTree = DT(feature_list = feature_list, max_depth = max_depth, min_split=min_split, min_gini=min_gini)
    TestTree.fit(training_data)
    prediction_list = TestTree.predict(test_data)
    actual_list = test_data.iloc[:,-1].to_list()
    for i in range(len(prediction_list)):
        if prediction_list[i] == actual_list[i]:
            correct_predictions += 1
    return correct_predictions / len(prediction_list)

def validate(training_data,validation_data,feature_list): # due to the high number of hyperparameters, this can be very slow. min_gini not validated and output not printed for brevity.
    best_acc = 0
    for max_depth in range(1,15):
        for min_split in range(1,7):
            # for min_gini in np.arange(0.0,0.2,0.02):
            acc = eval_acc(training_data,validation_data,feature_list,max_depth,min_split,0.08)
            if acc > best_acc:
                best_acc = acc
                best_depth = max_depth
                best_split = min_split
                best_gini = 0.08
    return best_depth,best_split,best_gini
                

In [13]:
best_hyperparameters_2 = validate(df2_train,df2_validate,feature_list_2)
print("Best hps for dataset 2:", best_hyperparameters_2)
print("Has an accuracy of:", eval_acc(df2_train,df2_test,feature_list_2,best_hyperparameters_2[0],best_hyperparameters_2[1],best_hyperparameters_2[2]))

Best hps for dataset 2: (4, 1, 0.08)
Has an accuracy of: 0.9261744966442953


In [ ]:
best_hyperparameters_1 = validate(df1_train,df1_validate,feature_list)
print("Best hps for dataset 1:", best_hyperparameters_1)
print("Has an accuracy of:", eval_acc(df1_train,df1_test,feature_list,best_hyperparameters_1[0],best_hyperparameters_1[1],best_hyperparameters_1[2]))